In [7]:
import av

input_container = av.open("rtsp://192.168.1.102/live/zoo1",timeout=5,  options={"fflags": "nobuffer", "flags": "low_delay"})
input_stream = input_container.streams.video[0]

In [3]:
import av
import cv2

# RTSP 流的 URL
rtsp_url = "rtsp://192.168.1.102/live/zoo1"

# 打开 RTSP 流
container = av.open(rtsp_url)
stream = container.streams.video[0]
# 遍历视频流中的帧
for frame in container.decode(stream):
    # 将帧转换为 numpy 数组
    img = frame.to_ndarray(format='bgr24')
    f = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # 使用 OpenCV 处理帧
    # 例如：显示帧
    cv2.imshow('Frame', f)
    
    # 按下 'q' 键退出
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 释放资源
cv2.destroyAllWindows()


In [ ]:
import cv2
from ultralytics import YOLO
import av
import subprocess as sp
import numpy as np

model = YOLO("yolo11n.pt", verbose=False)


# input_container = av.open("rtsp://192.168.1.107/live/zoo2")
# input_stream = input_container.streams.video[0]
# rate = input_stream.base_rate

cap = cv2.VideoCapture("rtsp://192.168.1.102/live/zoo1")
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))


freq = cv2.getTickFrequency()
prev_tick = cv2.getTickCount()

# width = input_stream.width
# height = input_stream.height
# fps = input_stream.base_rate

command = [
    'ffmpeg',
    '-y',
    '-f', 'rawvideo',
    '-vcodec', 'rawvideo',
    '-pix_fmt', 'bgr24',
    '-s', "{}x{}".format(width, height),
    '-r', str(fps),
    '-i', '-',
    '-c:v', 'libx264',
    '-pix_fmt', 'yuv420p',
    '-preset', 'veryfast',
    '-f', 'rtsp',
    '-rtsp_transport', 'tcp',
    '-g', '5',
    "rtsp://192.168.1.107/live/zoo-detect"
]
pipe = sp.Popen(command, stdin=sp.PIPE)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        raise IOError("Could not read frame from video stream.")
    
    results = model(frame, verbose=False)

    annotated_frame = results[0].plot()

    curr_tick = cv2.getTickCount()
    fps = freq / (curr_tick - prev_tick)
    prev_tick = curr_tick

    cv2.putText(annotated_frame, f"FPS: {int(fps)}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 1)


    pipe.stdin.write(annotated_frame.tobytes())



In [ ]:
import av

av.open('video=Iriun Webcam', format="dshow") 